# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Load and Read CSV
vacation_path = "clean_city_data.csv"
vacation_data = pd.read_csv(vacation_path)

#Had to remove extra column because previous index was added as a column
vacation_data = vacation_data[['City', 'Lat', 'Lng', 'Max Temp', 'Humidity',
                                  'Cloudiness', 'Wind Speed', 'Country', 'Date']]
vacation_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Valdivia,-39.81,-73.25,39.20,100,90,3.36,CL,1595866783
1,Hasaki,35.73,140.83,78.01,100,75,3.36,JP,1595866796
2,Petropavlovsk-Kamchatskiy,53.05,158.65,51.80,100,36,6.71,RU,1595866788
3,North Bend,43.41,-124.22,64.00,100,90,4.70,US,1595866805
4,Prince Rupert,54.32,-130.32,55.40,100,90,2.24,CA,1595866827
...,...,...,...,...,...,...,...,...,...
552,Riyadh,24.69,46.72,105.80,14,40,5.82,SA,1595866746
553,Khāsh,28.22,61.22,85.60,12,0,12.91,IR,1595866815
554,Agadez,19.75,10.25,106.27,10,86,8.30,NE,1595866801
555,Bāfq,31.61,55.41,94.78,10,1,13.82,IR,1595866797


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
# Configure gmaps
gmaps.configure(api_key= g_key)

# Store latitude and longitude in locations
locations = vacation_data[["Lat", "Lng"]].astype(float)

# Fill NaN values and convert to float
humidity = vacation_data['Humidity'].astype(float)

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)


# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=10)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Max temperature lower than 80 degrees but higher than 70
vacation_data = vacation_data.loc[(vacation_data['Max Temp'] <80) & (vacation_data['Max Temp'] >70)]

#Zero cloudiness
vacation_data = vacation_data.loc[(vacation_data['Cloudiness'] == 0)]

#Wind speed less than 10 mph
vacation_data = vacation_data.loc[(vacation_data['Wind Speed'] < 10)]

vacation_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
102,Tongzi,28.78,105.09,75.47,86,0,1.39,CN,1595866817
316,Havøysund,71.00,24.66,71.60,67,0,2.24,NO,1595866803
318,Benguela,-12.58,13.41,75.16,67,0,7.58,AO,1595866825
337,Athabasca,54.72,-113.29,71.01,66,0,3.13,CA,1595866824
465,Paraty,-23.22,-44.71,78.80,47,0,6.93,BR,1595866779
526,Zhezkazgan,47.80,67.71,77.65,25,0,8.32,KZ,1595866820
531,Bethanien,-26.50,17.16,70.77,22,0,9.84,NaN,1595866799
534,Lubango,-14.92,13.49,73.74,21,0,2.91,AO,1595866794
535,Zhangye,38.93,100.45,76.87,21,0,6.35,CN,1595866815
537,Vallenar,-28.57,-70.76,74.35,20,0,3.42,CL,1595866701


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#set empty lists
hotel = []
hotel_lat = []
hotel_lng = []

#define search
target_search = 'hotels'

#define radius
radius = 5000

hotel_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "keyword": target_search,
    "radius": radius,
    "key": g_key
}

for index, row in vacation_data.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params).json()
    
    try:
        hotel.append(response["results"][0]["name"])
        hotel_lat.append(response["results"][0]["geometry"]["location"]["lat"])
        hotel_lng.append(response["results"][0]["geometry"]["location"]["lng"])
    except (KeyError, IndexError):
        next

In [6]:
hotel_dict = {
    "Hotel Name": hotel,
    "Lat": hotel_lat,
    "Lng": hotel_lng
}

hotel_df = pd.DataFrame(hotel_dict)

hotel_df

,Hotel Name,Lat,Lng
0,Arctic View Glamping & Restaurant,71.020665,24.557998
1,Aparthotel Mil Cidades,-12.582186,13.409446
2,Days Inn by Wyndham Athabasca,54.719190,-113.249040
3,Carpe Diem Boutique Hotel,-23.215104,-44.713830
4,Hotel Zumrat,47.796575,67.675481
5,Bethanie Guesthouse,-26.501838,17.161983
6,Hotel Lubango,-14.920556,13.492989
7,Holiday Inn Express Zhangye,38.958800,100.434280
8,Hotel Origenes,-28.565958,-70.803930
9,Opuwo Country Lodge,-18.044682,13.833500


In [7]:
#Get city and country location of hotels
City = []
Country = []

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["address"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    
    response = requests.get(base_url, params).json()
    
    try:
        City.append(response["results"][0]["address_components"][2]["long_name"])
        Country.append(response["results"][0]["address_components"][3]["short_name"])
    except (KeyError, IndexError):
        next

In [8]:
#Turn City and Country to DataFrames
Country_df = pd.DataFrame(Country)
City_df = pd.DataFrame(City)

#Add City and Country Columns
hotel_df['City'] = City_df
hotel_df['Country'] = Country_df
hotel_df

,Hotel Name,Lat,Lng,City,Country
0,Arctic View Glamping & Restaurant,71.020665,24.557998,Troms og Finnmark,NO
1,Aparthotel Mil Cidades,-12.582186,13.409446,Benguela,Benguela
2,Days Inn by Wyndham Athabasca,54.719190,-113.249040,Athabasca,Division No. 13
3,Carpe Diem Boutique Hotel,-23.215104,-44.713830,Pontal,Paraty
4,Hotel Zumrat,47.796575,67.675481,Ulytau District,Karagandy Province
5,Bethanie Guesthouse,-26.501838,17.161983,Karas Region,NA
6,Hotel Lubango,-14.920556,13.492989,Lubango,Lubango
7,Holiday Inn Express Zhangye,38.958800,100.434280,Zhangye Shi,Gansu Sheng
8,Hotel Origenes,-28.565958,-70.803930,Vallenar,Vallenar
9,Opuwo Country Lodge,-18.044682,13.833500,Namibia,NaN


In [9]:
# base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    
# response = requests.get(base_url, params).json()
# response

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
locations

,Lat,Lng
0,71.020665,24.557998
1,-12.582186,13.409446
2,54.719190,-113.249040
3,-23.215104,-44.713830
4,47.796575,67.675481
5,-26.501838,17.161983
6,-14.920556,13.492989
7,38.958800,100.434280
8,-28.565958,-70.803930
9,-18.044682,13.833500


In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_template)

# Assign the marker layer to a variable
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))